# Views in Django

Views are one of the core components of Django's Model-View-Template (MVT) architecture. We will cover the following things in it:

1. What are Views?
2. Types of Views in Django.
3. Handling HTTP Methods (GET and POST).
4. Passing Data to Templates.
5. Creating Views for APIs (JSON Responses).
6. Class-Based Views (CBVs).
7. Testing APIs with curl and Postman.

---

## 1. What are Views?
A **View** in Django is a Python function or class that takes a web request and returns a web response. It acts as the bridge between the user's request and the output sent back by the server.

### Key Responsibilities of Views:
- Process the request.
- Perform any required logic.
- Return a response (HTML, JSON, etc.).

---

## 2. Types of Views in Django
Django provides two main ways to define views:

1. **Function-Based Views (FBVs)**: Views defined using Python functions.
2. **Class-Based Views (CBVs)**: Views defined using Python classes.

We will focus on **Function-Based Views** first and then explore **Class-Based Views**.

### Example: Function-Based View
Let's create a view in `playground` app that returns a webpage with a welcome message.

```python
# views.py
from django.http import HttpResponse

def home_view(request):
    return HttpResponse("<h1>Welcome to Django Views!</h1>")
```

### URL Mapping (App-level `urls.py`)
```python
# urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.home_view, name='home'),     # '' represents the root URL
]
```

### Main URL Mapping (`storefront/urls.py`)
```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('playground/', include('playground.urls')),
]
```

When you visit `http://localhost:8000/playground/`, you will see the response: **Welcome to Django Views!**

---

## 3. Handling HTTP Methods (GET and POST)
Web requests usually follow different HTTP methods. The most common are:

- **GET**: Used to request data from the server (e.g., loading a webpage).
- **POST**: Used to send data to the server (e.g., submitting a form).

### Example: Handling GET and POST Requests
Here is how to handle these methods in a Function-Based View:

```python
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt # To disable CSRF protection for this view, For now just ignore it.
def contact_view(request):
    if request.method == 'GET':
        return HttpResponse("<h1>Contact Us Form</h1><form method='POST'><input type='text' name='name' placeholder='Your Name'><button type='submit'>Submit</button></form>")
    elif request.method == 'POST':
        name = request.POST.get('name', 'Guest')
        return HttpResponse(f"<h1>Thank you, {name}, for reaching out!</h1>")
```

### URL Mapping (App-level `urls.py`)
```python
urlpatterns = [
    path('contact/', views.contact_view, name='contact'),
]
```

When you visit `http://localhost:8000/playground/contact/` using a browser (GET request), you will see the contact form. After submitting the form (POST request), you will see a thank-you message.

---

## 4. Passing Data to Templates
Views often need to send data to templates for rendering dynamic content. Templates are HTML files that can include placeholders for data.

### Example: Passing Context Data to Templates

```python
from django.shortcuts import render

def profile_view(request):
    # Data to be passed to the template
    context = {
        'name': 'Alice',
        'profession': 'Data Scientist',
        'skills': ['Python', 'Django', 'Machine Learning', 'MySQL']
    }
    return render(request, 'profile.html', context)
```

### Template File: `profile.html`
Create this file in the `templates` folder of your app.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Profile</title>
</head>
<body>
    <h1>Welcome, {{ name }}!</h1>
    <p>Profession: {{ profession }}</p>
    <h2>Skills:</h2>
    <ul>
        {% for skill in skills %}
        <li>{{ skill }}</li>
        {% endfor %}
    </ul>
</body>
</html>
```

### URL Mapping (App-level `urls.py`)
```python
urlpatterns = [
    path('', views.home_view, name='home'),
    path('contact/', views.contact_view, name='contact'),
    path('profile/', views.profile_view, name='profile'),
]
```

When you visit `http://localhost:8000/playground/profile/`, you will see a dynamically generated webpage based on the `context` dictionary.

---

## 5. Creating Views for APIs (JSON Responses)
Most of the times, we need to return data in JSON format instead of rendering an HTML page. This is useful for creating APIs.

### Example: Returning JSON Response

```python
from django.http import JsonResponse

def product_api_view(request):
    products = [
        {'id': 1, 'name': 'Laptop', 'price': 1200},
        {'id': 2, 'name': 'Smartphone', 'price': 800},
        {'id': 3, 'name': 'Tablet', 'price': 500}
    ]
    return JsonResponse({'status': 'success', 'products': products})
```

### URL Mapping (App-level `urls.py`)
```python
urlpatterns = [
    ...
    path('products/', views.product_api_view, name='product_api'),
]
```

When you visit `http://localhost:8000/playground/products/`, you will see the JSON response:

```json
{
    "status": "success",
    "products": [
        {"id": 1, "name": "Laptop", "price": 1200},
        {"id": 2, "name": "Smartphone", "price": 800},
        {"id": 3, "name": "Tablet", "price": 500}
    ]
}
```

---

## 6. Class-Based Views (CBVs)
Class-Based Views provide an object-oriented way to define views in Django. They allow reusability and a cleaner structure. 

### Example: Class-Based View
For the `ProductAPIView` class, we will define separate methods for handling GET and POST requests. 
(CSRF protection is disabled for this view for now so that we can test it easily.)
```python
from django.http import JsonResponse
from django.views import View
from django.views.decorators.csrf import csrf_exempt
from django.utils.decorators import method_decorator

@method_decorator(csrf_exempt, name='dispatch')     # To disable CSRF protection for this view, For now just ignore it.
class ProductAPIView(View):
    def get(self, request):
        products = [
            {'id': 1, 'name': 'Laptop', 'price': 1200},
            {'id': 2, 'name': 'Smartphone', 'price': 800},
            {'id': 3, 'name': 'Tablet', 'price': 500}
        ]
        return JsonResponse({'status': 'success', 'products': products})

    def post(self, request):
        import json
        body = json.loads(request.body)
        name = body.get('name')
        price = body.get('price')
        return JsonResponse({'status': 'success', 'message': f'Product {name} with price {price} added successfully!'})
```

### URL Mapping (App-level `urls.py`)
```python
from .views import ProductAPIView

urlpatterns += [
    path('products/class/', ProductAPIView.as_view(), name='product_api_class'),
]
```

---

## 7. Testing APIs with curl and Postman

### Using `curl` to Test GET
To test the **GET** method of our `ProductAPIView`, use the following `curl` command:

```bash
curl -X GET http://localhost:8000/playground/products/class/
```

### Using `curl` to Test POST
To test the **POST** method, use the following `curl` command:

```bash
curl -X POST http://localhost:8000/playground/products/class/ \
     -H "Content-Type: application/json" \
     -d '{"name": "Headphones", "price": 150}'
```

You will receive a response like:

```json
{
    "status": "success",
    "message": "Product Headphones with price 150 added successfully!"
}
```

### Using Postman
[Postman](https://www.postman.com/downloads/) is a popular API client that allows you to test APIs easily. You can use it to test both GET and POST requests. From our next tutorial, we will use Postman to test APIs. There is `Thunder Client` extension in VS Code as well that we can use.

There is a lot more to explore in Django views, especially in Class-Based Views. We will cover that in later of the course.

